In [1]:
!pip install razdel -q
!pip install nltk -q

# Домашнее задание № 2. Мешок слов

In [2]:
import razdel
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [ ]:
data = pd.read_csv('labeled.csv')

train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
# дефолтная токенизация
vectorizer = TfidfVectorizer(min_df=5, max_df=0.4)

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [ ]:
clf = LogisticRegression()
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.82      0.96      0.89       945
         1.0       0.89      0.60      0.72       497

    accuracy                           0.84      1442
   macro avg       0.86      0.78      0.80      1442
weighted avg       0.85      0.84      0.83      1442



In [ ]:
def tokenizer(text):
    tokenized_text = []
    for word in razdel.tokenize(text):
        tokenized_text.append(word.text)
    return tokenized_text

In [ ]:
# razdel токенизация
vectorizer = TfidfVectorizer(tokenizer=tokenizer, min_df=5, max_df=0.4)

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
clf = LogisticRegression()
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.82      0.95      0.88       945
         1.0       0.86      0.60      0.71       497

    accuracy                           0.83      1442
   macro avg       0.84      0.78      0.80      1442
weighted avg       0.84      0.83      0.82      1442



Метрики получились практически одинаковыми, но у razdel все-таки немного меньше.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [6]:
data = pd.read_csv('labeled.csv')

train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [7]:
russian_stopwords = stopwords.words('russian')

In [ ]:
# логистическая регрессия с tf-idf векторайзером
vectorizer = TfidfVectorizer(min_df=5, max_df=0.4, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [ ]:
clf = LogisticRegression(penalty='l2', class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.89      0.85      0.87       947
         1.0       0.74      0.80      0.77       495

    accuracy                           0.83      1442
   macro avg       0.81      0.83      0.82      1442
weighted avg       0.84      0.83      0.84      1442



In [ ]:
vectors = vectorizer.transform(data.comment)
probas = clf.predict_proba(vectors)
text_proba = {}
for text, proba in zip(data.comment, probas):
    text_proba[text] = proba[1]

cnt = Counter(text_proba)
cnt.most_common(10)

[('Нахуй иди чмо ебаное, рот твой ебал. Говна поешь, быдло\n',
  0.9973361631087458),
 ('Нахуй иди, я тебе весь тред что ли читать буду? Пидор, бешбармак тебе в хычин!\n',
  0.9965190458227453),
 ('АЛЛО БЛЯДЬ АБУ ТЫ Ч ОХУЕЛ? ВЫГОНЯЙ НАХУЙ СВОЕГО ПОДЗАЛУПНОГО ДРУЖКА ВАРЛАМОВА НАХУЙ ИЗ Б ЭТА ХУЙНЯ ЗАЕБАЛА В 5 ТРЕДОВ БЛЯДЬ НА ГЛАВНОЙ ВИСЕТЬ, БАНЫЙ ТВОЙ РОТ ДЫРЯВЫЙ БЛЯДЬ, ПРОДАЖНАЯ МРАЗЬ!\n',
  0.9960805712922417),
 ('ИДИ НАХУЙ, СУКА, ЗАВАЛИ ЕБАЛО, БЛЯТЬ\n', 0.9949596706615794),
 ('Хохлы-объебосы порвались, лол\n', 0.9939471429982177),
 ('Ну оль, ну вот че тебе надо? Иди в по сри, а этот достопочтенный тред оставь в покое\n',
  0.993914966366831),
 ('Так воюют Русские А так воюют хохлы\n', 0.9938457427661792),
 ('Блядь абу нахуй ссылай этих дегенератов в фаг, всем похуй на их шлюх\n',
  0.9935381495097249),
 ('Ебать вы тупые дебилы, ой блять\n', 0.9926436967666499),
 ('Шмароёб реально задолбал. Зачем он срёт этой шалавой? В одиночестве пусть шышку хоть до основания стирает, но нахуй она на

In [ ]:
# наивный байесовский классификатор с count векторайзером
vectorizer = CountVectorizer(min_df=3, max_df=0.4, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [ ]:
clf = MultinomialNB(alpha=1, force_alpha=True)
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.90      0.88       947
         1.0       0.79      0.71      0.75       495

    accuracy                           0.84      1442
   macro avg       0.82      0.81      0.81      1442
weighted avg       0.83      0.84      0.83      1442



In [ ]:
vectors = vectorizer.transform(data.comment)
probas = clf.predict_proba(vectors)
text_proba = {}
for text, proba in zip(data.comment, probas):
    text_proba[text] = proba[1]

cnt = Counter(text_proba)
cnt.most_common(10)

[('Возьмём как пример Россию, западноевропейские страны и США. Идёт метисация, сознательная политика замещения белого населения на пришлое черно-коричневое. Идёт создание новой расы метисов, исламизация и почернение. В крупных городах половина населения - выходцы из ебеней Мексики, Африки, Ближнего Востока, а в случае с Россией - Кавказа и Средней Азии. Этнические ниггеро-арабские гетто верят на хую законы как хотят, чудовищная по масштабам этническая преступность. Говорить о миграции и тем более затрагивать тему замещения коренного населения властями нельзя, иначе бутылка. Свобода слова тут не для вас, молодой человек. При этом говорить о том, что белые должны вымереть, и это нормально - можно. Белые официально вымирают ведётся пропаганда так или иначе направленная на снижение рождаемости белого населения. Феминизм, ЛГБТ, чайлдфри. Каждая женщина в Швеции - леволиберальная феминистка, это страна победившего феминизма. Что сегодня там происходит - страшно делается. Пропагандируются сме

Кажется, все тексты отличаются, и все тексты токсичные. У второго классификатора все самые токсичные тексты получились длиннее.

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов.

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [ ]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.4, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [ ]:
# логистическая регрессия
clf = LogisticRegression(penalty='l2', class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.89      0.85      0.87       947
         1.0       0.74      0.80      0.77       495

    accuracy                           0.83      1442
   macro avg       0.81      0.83      0.82      1442
weighted avg       0.84      0.83      0.84      1442



In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_importance = np.abs(clf.coef_[0])
top_indices = feature_importance.argsort()[-5:][::-1]
[feature_names[i] for i in top_indices]

['хохлы', 'хохлов', 'тебе', 'дебил', 'блядь']

In [10]:
vectorizer = CountVectorizer(max_df=0.01, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [11]:
# дерево решений
clf = DecisionTreeClassifier(max_depth=2000, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.87      0.79      0.83       990
         1.0       0.62      0.74      0.67       452

    accuracy                           0.78      1442
   macro avg       0.74      0.77      0.75      1442
weighted avg       0.79      0.78      0.78      1442



In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_importance = clf.feature_importances_
top_indices = feature_importance.argsort()[-5:][::-1]
[feature_names[i] for i in top_indices]

['очень', 'тебе', 'хохлы', 'нахуй', 'хохлов']

In [ ]:
vectorizer = CountVectorizer(max_features=10000, max_df=0.1, stop_words=russian_stopwords, ngram_range=(1, 3))

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [ ]:
# наивный байес
clf = MultinomialNB(alpha=1, force_alpha=True)
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.85      0.90      0.88       947
         1.0       0.79      0.71      0.75       495

    accuracy                           0.83      1442
   macro avg       0.82      0.80      0.81      1442
weighted avg       0.83      0.83      0.83      1442



In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_importance = np.abs(clf.feature_log_prob_[1])
top_indices = feature_importance.argsort()[-5:][::-1]
[feature_names[i] for i in top_indices]

['ёмкости', 'отправила', 'очень дорого', 'оценки', 'охраны']

In [ ]:
vectorizer = CountVectorizer(max_features=10000, max_df=0.1, stop_words=russian_stopwords, ngram_range=(1, 3))

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [ ]:
# рандомный лес
clf = RandomForestClassifier(max_depth=1000, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.81      0.84       947
         1.0       0.68      0.75      0.71       495

    accuracy                           0.79      1442
   macro avg       0.77      0.78      0.77      1442
weighted avg       0.80      0.79      0.79      1442



In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_importance = clf.feature_importances_
top_indices = feature_importance.argsort()[-5:][::-1]
[feature_names[i] for i in top_indices]

['очень', 'хохлы', 'тебе', 'хохлов', 'нахуй']